In [2]:
%run Classes/SPDEs.ipynb
%run Classes/Rule.ipynb
%run Classes/Model.ipynb
%run Classes/Noise.ipynb
%run Classes/full_visualization.ipynb
# %run Algorithms/Algorithm_2.ipynb
%run Algorithms/Algorithm_2_FFT.ipynb
%run Classes/Burgers_PDE_FIND.ipynb

#Disable progress bars from tqdm
from functools import partialmethod
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

In [43]:
#Set global parameters

a, b, s, t = 0, 16, 0, 10 # space-time boundaries

X_, T_ = np.linspace(a,b,1025), np.linspace(s,t,2001) # space-time grid for generating solutions
x_space = 2
t_space = 10
X, T = X_[::x_space], T_[::t_space] # space-time grid of observed points. O_X and O_T

eps = 0.1 #Viscosity

k = 120 # Number of realizations (multiple of 3)
m = 20 # Number of test cases, m < k

N = 10 # Number of experiments

noise = 0.0 #noise intensity

#Model parameters
height = 3
deg = 5
kernel_deg = 2
noise_deg = -1.5
free_num = 2
BC = 'P'

R = Rule(kernel_deg = kernel_deg,
         noise_deg = noise_deg,
         free_num = free_num) # initialize rule. No multiplicative width and forcing. 

#Polynomial interpolation parameters for PDE-FIND
deg_x = 5
deg_t = 5

true_eps = False #whether to use true value of eps for PDE-FIND
finite_diff = True #whether to use finite difference (vs poly interpolation) for PDE-FIND

offset_x = 30
offset_t = 30

time_scale = 10 # increase the time interval for finer Euler scheme in PDE-FIND

In [44]:
%%time

exps_Alg2 = []
exps_v = []
exps_v_coef = []
exps_find = []
exps_find_eps = []
exps_find_lambd = []

for n in range(0,N):
    # Create realizations of initial conditions. 
    IC1 = Noise().initial(k//3, X_, scaling = 1) # one cycle
    IC2 = Noise().initial(k//3, X_, scaling = 2) # two cycles
    IC4 = Noise().initial(k//3, X_, scaling = 4) # four cycles

    IC = np.concatenate([IC1, IC2, IC4])

    W = np.zeros((k, len(T_), len(X_))) # No forcing
    
    #Generate solutions
    Burgers_ = SPDE(BC = BC, IC = IC, eps = eps).Burgers_FFT(W, T = T_, X = X_) # solve Burger's equations on the fine grid

    Burgers = Burgers_[:,::t_space,::x_space] # Observed points of Burger's equations
    
    Burgers = np.float32(Burgers)
    
    # Translate solution to a dataframe for the visualization class
    Burgers_true = [pd.DataFrame(B, index = T, columns = X) for B in Burgers]
    
    #
    test = np.random.choice(np.arange(k), m, replace = False) # choose test set
        
       
    Burgers_noise = np.array([B+np.random.normal(scale = noise*np.abs(B).mean().mean(), size = (len(T),len(X))) for B in Burgers_true])
    Burgers_noise = Burgers_noise.astype('float32')
    for i in range(k):
        Burgers_noise[i, :, -1] = Burgers_noise[i, :, 0]
    
    
    #ALGORITHM 2
    
    experiment_noise = IML(Burgers_noise,
                           Rule = R,
                           eps = eps,
                           height = height,
                           deg = deg,
                           T = T,
                           X = X,
                           step = 1) # initialize experiment for algorithm 2
    experiment_noise.set_train_test(test = test) # initialize test set

    experiment_noise.set_trees() # extract trees

    experiment_noise.training_models(time_grid_num = 10) # Create models for Step 2 of Algorithm 2
    
    
    experiment_noise.fit_training_model() # Linear fit for Step 2 of Algorithm 2
    experiment_noise.learn_with_fitted() # Recurcive steps 3 & 4 of Algorithm 2
    
    
    # Predicted values on the noisy data. Outliers that blow up during the experiment are excluded

    Prediction_noise = experiment_noise.to_df_list()
    show_prediction = summary(test)

    outliers = []
    #outliers = [i for i in range(len(test)) if show_prediction.one_error(i, Burgers_true, Prediction_noise) > 1]
    test_no_outliers = [test[i] for i in range(len(test)) if i not in outliers]
    Prediction_noise = [Prediction_noise[i] for i in range(len(test)) if i not in outliers]
    
    print('Numerical Scheme for Algorithm 2 failed for {} equation with the above coefficients.'.format(len(outliers)))
    
    # Errors
    show_no_outliers = summary(test_no_outliers)
    er = show_no_outliers.errors(Burgers_true, Prediction_noise, show = False, full = False)
    exps_Alg2.append(show_no_outliers)
    
    
    
    #PDE-FIND
    
    # PDE-FIND with polynomial interpolation on the noisy data

    # Number of equations used for the PDE-FIND algorithm
    learning_samples = k-m

    u = np.array([Burgers_noise[i] for i in range(len(Burgers_noise)) if i not in test])

    dt, dx = T[1]-T[0], X[1]-X[0]
    # get viscosity and non linear coeficient of the Burgers equation
    # Use polynomial interpolation to differentiate the noisy data
    if finite_diff: eps_find, lambd_find = PDE_FIND_Burgers(u[:learning_samples], dt, dx, diff='finite_diff')
    else: eps_find, lambd_find = PDE_FIND_Burgers(u[:learning_samples],
                                                    dt,
                                                    dx,
                                                    diff='poly',
                                                    offset_x = offset_x,
                                                    offset_t = offset_t,
                                                    deg_x = deg_x,
                                                    deg_t = deg_t)
    exps_find_eps.append(eps_find)
    exps_find_lambd.append(lambd_find)

    S_ = np.linspace(T[0] , T[-1], time_scale*(len(T)-1)+1)

    IC_ = np.array([B[0] for B in Burgers_noise])

    W_ = np.zeros((k, len(T_), len(X)))

    if true_eps: eps_ = eps
    else: eps_ = eps_find

    Burgers_noise_FIND = SPDE(BC = 'P',
                              IC = IC_,
                              eps = eps_).Burgers_FFT(W_,
                                                  lambd = -lambd_find,
                                                  T = S_,
                                                  X = X)
                                                      
                                                       
    
    Burgers_noise_FIND = [pd.DataFrame(Burgers_noise_FIND[i, ::time_scale, :], index = T, columns = X) for i in test]
    
    #test_without_nan = test
    test_without_nan = [test[i] for i in range(len(test)) if not Burgers_noise_FIND[i].isnull().values.any()]
    Burgers_noise_FIND_ = [B for B in Burgers_noise_FIND if not B.isnull().values.any()]

    print('Numerical Scheme for PDE-FIND failed for {} equation with the above coefficients.'.format(len(test)-len(test_without_nan)))

    
    show_FIND = summary(test_without_nan)

    er_noise = show_FIND.errors(Burgers_true, Burgers_noise_FIND_, show = False, full = False)
    exps_find.append(show_FIND)
    


Creating Model
Numerical Scheme for Algorithm 2 failed for 0 equation with the above coefficients.
Numerical Scheme for PDE-FIND failed for 0 equation with the above coefficients.
Creating Model
Numerical Scheme for Algorithm 2 failed for 0 equation with the above coefficients.
Numerical Scheme for PDE-FIND failed for 0 equation with the above coefficients.
Creating Model
Numerical Scheme for Algorithm 2 failed for 0 equation with the above coefficients.
Numerical Scheme for PDE-FIND failed for 0 equation with the above coefficients.
Creating Model
Numerical Scheme for Algorithm 2 failed for 0 equation with the above coefficients.
Numerical Scheme for PDE-FIND failed for 0 equation with the above coefficients.
Creating Model
Numerical Scheme for Algorithm 2 failed for 0 equation with the above coefficients.
Numerical Scheme for PDE-FIND failed for 0 equation with the above coefficients.
Creating Model
Numerical Scheme for Algorithm 2 failed for 0 equation with the above coefficients.
N

In [72]:
for exps in [exps_Alg2, exps_find]:

    print("Average average error:", np.mean([e.av_er for e in exps]))
    print("Maximum average error:", max([e.av_er for e in exps]))
    print("Minimum average error:", min([e.av_er for e in exps]))
    print('')

    print("Average maximum error:", np.mean([e.max_er for e in exps]))
    print("Maximum maximum error:", max([e.max_er for e in exps]))
    print("Minimum maximum error:", min([e.max_er for e in exps]))
    print('')

    print("Average minimum error:", np.mean([e.min_er for e in exps]))
    print("Maximum minimum error:", max([e.min_er for e in exps]))
    print("Minimum minimum error:", min([e.min_er for e in exps]))
    print('')

    print("=====================")

Average average error: 0.2239173196649292
Maximum average error: 2.018596246693517
Minimum average error: 0.0008716690024409078

Average maximum error: 1.6769330915949232
Maximum maximum error: 13.014984894053104
Minimum maximum error: 0.002080609583293296

Average minimum error: 0.0007650899709596656
Maximum minimum error: 0.003999984263644648
Minimum minimum error: 9.956414452728521e-05

Average average error: 0.017157013549694906
Maximum average error: 0.026880999944501195
Minimum average error: 0.005965225118296762

Average maximum error: 0.053758763204256946
Maximum maximum error: 0.10867721447684361
Minimum maximum error: 0.01604159223295464

Average minimum error: 0.003288343238769651
Maximum minimum error: 0.0057935841878649745
Minimum minimum error: 0.0010323092613782703



In [46]:
print("Average viscosity: {}, lambda: {}".format(np.mean(exps_find_eps), np.mean(exps_find_lambd)))
print("Maximum viscosity: {}, lambda: {}".format(max(exps_find_eps), max(exps_find_lambd)))
print("Minimum viscosity: {}, lambda: {}".format(min(exps_find_eps), min(exps_find_lambd)))

Average viscosity: 0.11898150223863317, lambda: -0.9878175670194862
Maximum viscosity: 0.12922896508573053, lambda: -0.9737782587435511
Minimum viscosity: 0.11102494383857071, lambda: -0.9924759344481009


In [71]:
K = {5,8}
for exps in [[exps_Alg2[k] for k in range(10) if k not in K], [exps_find[k] for k in range(10) if k not in K]]:

    print("Average average error:", np.mean([e.av_er for e in exps]))
    print("Maximum average error:", max([e.av_er for e in exps]))
    print("Minimum average error:", min([e.av_er for e in exps]))
    print('')

    print("Average maximum error:", np.mean([e.max_er for e in exps]))
    print("Maximum maximum error:", max([e.max_er for e in exps]))
    print("Minimum maximum error:", min([e.max_er for e in exps]))
    print('')

    print("Average minimum error:", np.mean([e.min_er for e in exps]))
    print("Maximum minimum error:", max([e.min_er for e in exps]))
    print("Minimum minimum error:", min([e.min_er for e in exps]))
    print('')

    print("=====================")

Average average error: 0.004621144089675013
Maximum average error: 0.024759779748431042
Minimum average error: 0.0008716690024409078

Average maximum error: 0.016930145916709806
Maximum maximum error: 0.09106565603453168
Minimum maximum error: 0.002080609583293296

Average minimum error: 0.000735927230480257
Maximum minimum error: 0.003999984263644648
Minimum minimum error: 9.956414452728521e-05

Average average error: 0.01673840561095561
Maximum average error: 0.026880999944501195
Minimum average error: 0.005965225118296762

Average maximum error: 0.05409432189632494
Maximum maximum error: 0.10867721447684361
Minimum maximum error: 0.01604159223295464

Average minimum error: 0.0028862514307403412
Maximum minimum error: 0.004677630311192571
Minimum minimum error: 0.0010323092613782703



In [67]:
exps_Alg2[5].error_list

[0.0031742280727352357,
 0.0027738661350978984,
 0.0011391455704475523,
 0.0027636723721892018,
 0.002623507380202401,
 0.005821907167080326,
 10.504957316486482,
 0.017269730852147894,
 0.0017297188700348355,
 0.0631502780354008,
 12.006201369360152,
 13.014984894053104,
 0.004545341317388923,
 4.7079147361262725,
 0.01696177965370958,
 0.0025677566192509184,
 0.004519920527451801,
 0.0024768166484147253,
 0.004769717751739776,
 0.0015792308710345132]

In [69]:
exps_Alg2[8].error_list

[0.00134322900869213,
 0.0023225003487707422,
 0.003633534798451749,
 0.004808361935723348,
 0.0059972324805139775,
 0.006262054974366487,
 0.0008991772429137162,
 0.002663918229103143,
 0.0006243362953070477,
 0.001297372873600585,
 0.0029191170013166496,
 3.618904854562451,
 0.0011388553340861299,
 0.0009094055448545618,
 0.002432226666138315,
 0.0016008994011953335,
 0.0043004270151504155,
 0.004238149493141338,
 0.0051862231277614605,
 0.0006740684339616821]

In [73]:
exps_Alg2[6].error_list

[0.0014761848295323985,
 0.0025441568831565697,
 0.002336394396734921,
 0.002811231879735304,
 0.0008481723404014112,
 0.003099896381744059,
 0.0016579433182324298,
 0.0007054061102531823,
 0.0017260607259161152,
 0.010466699936685819,
 0.0005207001560206798,
 0.0037617268661389856,
 0.0021027817121311064,
 0.000650973129420816,
 0.0008085207645555947,
 0.0018030957793219182,
 0.0009440378011368112,
 0.009520858683816445,
 0.0005913438482277973,
 0.0009760946844685452]

In [74]:
exps_find[6].error_list

[0.024282721887714992,
 0.049243447127595985,
 0.013345710280536,
 0.00971454517579864,
 0.0055370951143931865,
 0.021589313000175747,
 0.01021387061796582,
 0.013297356163619841,
 0.016762805516430977,
 0.0525460857008907,
 0.010336179071284984,
 0.013668393252971558,
 0.01926259576113366,
 0.012447455836770403,
 0.015249722759090454,
 0.017744997682418846,
 0.004777541983929848,
 0.0753868012079021,
 0.004677630311192571,
 0.013194155833573248]

In [78]:
list_of_errors_Alg2 = []
list_of_errors_find = []

for k in range(10):
    list_of_errors_Alg2 += exps_Alg2[k].error_list
    list_of_errors_find += exps_find[k].error_list

list_of_errors_Alg2 = np.array([err for err in list_of_errors_Alg2 if err < 1])
list_of_errors_find = np.array([err for err in list_of_errors_find if err < 1])

print('Number of equations that failed for Algorithm 2:', m*N - len(list_of_errors_Alg2))
print('')
print('Avg error for Algorithm 2:', list_of_errors_Alg2.mean())
print('Max error for Algorithm 2:', list_of_errors_Alg2.max())
print('Min error for Algorithm 2:', list_of_errors_Alg2.min())

print('========================================')
print('')
print('Number of equations that failed for PDE-FIND:', m*N - len(list_of_errors_find))
print('')
print('Avg error for PDE-FIND:', list_of_errors_find.mean())
print('Max error for PDE-FIND:', list_of_errors_find.max())
print('Min error for PDE-FIND:', list_of_errors_find.min())

Number of equations that failed for Algorithm 2: 5

Avg error for Algorithm 2: 0.004771798781525011
Max error for Algorithm 2: 0.09106565603453168
Min error for Algorithm 2: 9.956414452728521e-05

Number of equations that failed for PDE-FIND: 0

Avg error for PDE-FIND: 0.017157013549694906
Max error for PDE-FIND: 0.10867721447684361
Min error for PDE-FIND: 0.0010323092613782703
